In [1]:
import pandas as pd
import pickle
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge

from sklearn.metrics import mean_squared_error, root_mean_squared_error

In [3]:
import psutil

#Getting % usage of virtual memory(3rd field)
print ('RAM memory % used:', psutil.virtual_memory()[2])
#Getting usage of virtual_memory in GB (4th field)
print('RAM used (GB):', psutil.virtual_memory()[3]/1000000000)

#cpu usage %%
print('The CPU usage is: ', psutil.cpu_percent(4))

RAM memory % used: 36.7
RAM used (GB): 2.717601792
The CPU usage is:  9.6


In [ ]:
pd.__version__

In [4]:
df = pd.read_parquet('../data/yellow_tripdata_2023-01.parquet')


In [5]:
df.shape

(3066766, 19)

In [6]:
df.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee
0,2,2023-01-01 00:32:10,2023-01-01 00:40:36,1.0,0.97,1.0,N,161,141,2,9.3,1.00,0.5,0.00,0.0,1.0,14.30,2.5,0.00
1,2,2023-01-01 00:55:08,2023-01-01 01:01:27,1.0,1.10,1.0,N,43,237,1,7.9,1.00,0.5,4.00,0.0,1.0,16.90,2.5,0.00
2,2,2023-01-01 00:25:04,2023-01-01 00:37:49,1.0,2.51,1.0,N,48,238,1,14.9,1.00,0.5,15.00,0.0,1.0,34.90,2.5,0.00
3,1,2023-01-01 00:03:48,2023-01-01 00:13:25,0.0,1.90,1.0,N,138,7,1,12.1,7.25,0.5,0.00,0.0,1.0,20.85,0.0,1.25
4,2,2023-01-01 00:10:29,2023-01-01 00:21:19,1.0,1.43,1.0,N,107,79,1,11.4,1.00,0.5,3.28,0.0,1.0,19.68,2.5,0.00


In [7]:
df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

df.duration.std()

42.594351241920904

In [8]:
len(df[(df.duration >= 1) & (df.duration <= 60)]) *100/ len(df)

98.1220282212598

In [9]:
df= df[(df.duration >= 1) & (df.duration <= 60)]

In [10]:
categorical = ['PULocationID', 'DOLocationID']

df_train = df[categorical].astype(str)
y_train= df['duration']

In [11]:
del df

In [12]:
train_dicts = df_train.to_dict(orient='records')

In [13]:
train_dicts

[{'PULocationID': '161', 'DOLocationID': '141'},
 {'PULocationID': '43', 'DOLocationID': '237'},
 {'PULocationID': '48', 'DOLocationID': '238'},
 {'PULocationID': '138', 'DOLocationID': '7'},
 {'PULocationID': '107', 'DOLocationID': '79'},
 {'PULocationID': '161', 'DOLocationID': '137'},
 {'PULocationID': '239', 'DOLocationID': '143'},
 {'PULocationID': '142', 'DOLocationID': '200'},
 {'PULocationID': '164', 'DOLocationID': '236'},
 {'PULocationID': '141', 'DOLocationID': '107'},
 {'PULocationID': '234', 'DOLocationID': '68'},
 {'PULocationID': '79', 'DOLocationID': '264'},
 {'PULocationID': '164', 'DOLocationID': '143'},
 {'PULocationID': '138', 'DOLocationID': '33'},
 {'PULocationID': '33', 'DOLocationID': '61'},
 {'PULocationID': '79', 'DOLocationID': '186'},
 {'PULocationID': '90', 'DOLocationID': '48'},
 {'PULocationID': '113', 'DOLocationID': '255'},
 {'PULocationID': '237', 'DOLocationID': '239'},
 {'PULocationID': '143', 'DOLocationID': '229'},
 {'PULocationID': '137', 'DOLocat

In [14]:
dv = DictVectorizer()
X_train = dv.fit_transform(train_dicts)



In [15]:
X_train.shape

(3009173, 515)

In [16]:
y_train = y_train.values

lr = LinearRegression()
lr.fit(X_train, y_train)

y_pred = lr.predict(X_train)

mean_squared_error(y_train, y_pred, squared=False)

/home/codespace/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


7.649261027792376

In [33]:
root_mean_squared_error(y_train, y_pred)

7.649261027792376

In [18]:
#save as pickle
with open('model_pkl', 'wb') as f_out:
    pickle.dump((dv, lr), f_out)

In [2]:
#load pickle
with open('model_pkl' , 'rb') as f:
    dv, lr = pickle.load(f)

In [3]:
import psutil

#Getting % usage of virtual memory(3rd field)
print ('RAM memory % used:', psutil.virtual_memory()[2])
#Getting usage of virtual_memory in GB (4th field)
print('RAM used (GB):', psutil.virtual_memory()[3]/1000000000)

#cpu usage %%
print('The CPU usage is: ', psutil.cpu_percent(4))

RAM memory % used: 36.8
RAM used (GB): 2.73195008
The CPU usage is:  5.7


In [4]:
df = pd.read_parquet('../data/yellow_tripdata_2023-02.parquet')

df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)
df= df[(df.duration >= 1) & (df.duration <= 60)]
categorical = ['PULocationID', 'DOLocationID']

df_val = df[categorical].astype(str)
y_val= df['duration']
del df
val_dicts = df_val.to_dict(orient='records')

X_val = dv.transform(val_dicts)

y_val = y_val.values

y_val_pred = lr.predict(X_val)

mean_squared_error(y_val, y_val_pred, squared=False)

/home/codespace/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


7.811832836304415